# Práctica 6 Arboles binarios
Juan C. Agui  
ICAI IMAT Abril 2024

**Aviso**  
En este notebook, utilizamos el paquete de clases dinámicas jdc ( https://pypi.org/project/jdc/ ) instalar con  "pip install jdc" que nos permite añadir métodos a clases previamente definidos en otra celda anterior del mismo notebook.  De esta manera podemos ir comentando y enriqueciendo las clases de forma gradual, que debe ser más inteligible, que no el código en su conjunto.

Para su uso, hay que empezar la celda con la directiva : "%%add_to ClassName"   y los métodos definidos en esta celda se añaden a la clase ClassName.

Hagamos unos import

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import datetime 
import jdc

## Definición Clases básicas

## Clase Nodo

In [ ]:

# definición de clase nodo, con algunas funciones de utilidad...
class Node():
    def __init__(self,  key=None, value=None):
        # pointer to parent
        self.parent  = None 
        # pointer to left child  
        self.left    = None 
        # pointer to right child
        self.right   = None 
        # key that identifies the node
        self.key     = key    
        # additional data
        self.value    = value   # additional data
    
    def __str__(self):
        return f'Node {self.key=}, {self.value=} id={id(self)}'

    def __repr__(self):
        return f'Node key={self.key}'

    def __lt__(self,other):
        return self.key < other.key
    
    def show(self):
        parentId = self.parent if self.parent else "N/A"
        leftId = self.left.key if self.left else "N/A"
        rightId = self.right.key if self.right else "N/A"
        return f"Node Id={self.key}  Parent={parentId} Left={leftId} Right={rightId}, Height={self.height()}"
    
    def isLeaf(self):
        if self.left or self.right:
            return False
        else:
            return True
    def isOrphan(self):
        return self.parent is None
    
    def isLeftChild(self):
        if self.isOrphan():
            return False
        
        if self.parent.left == self:
            return True
        else:
            return False
    
    def isRightChild(self):
        if self.isOrphan():
            return False
            
        if self.parent.right == self: 
            return True
        else:
            return False
        
    def height(self):
        if self.isLeaf():
            return 0
        else:
            l_height = self.left.height() if self.left else 0
            r_height = self.right.height() if self.right else 0
            return 1 + max(l_height, r_height)
        
    # retorna el skew o desquilibrio de un nodo, mirando a la altura de sus hijos...
    def skew(self):
        l_height = self.left.height()  + 1 if self.left  else 0
        r_height = self.right.height() + 1 if self.right else 0
        return r_height - l_height
    
    def connectAsLeftChildOfNode(self,parent):
        """ sets self as left child of parent"""
        self.parent = parent
        parent.left = self
        
    def connectAsRightChildOfNode(self,parent):
        """ sets self as right child of parent"""
        self.parent = parent
        parent.right = self

print ("Node class updated")

## Clase BST, Binary Search Tree
Aquí definimos lo básico del BST y un método plot que nos permite visualizarlo fácilmente.... Luego lo vamos enriqueciendo progresivamente en celdas posteriores

In [ ]:
      
class BST():
    def __init__(self, rootNode=None):
        self.rootNode = rootNode
    
    def __str__(self):
        """   printable info of a tree  """
        return f'Tree with root Node key = {self.rootNode.key} and Height= {self.height()} Size = {self.size()} Skew = {self.skew()}'
        
    def height(self):
        """   returns height of a tree, 0  if empty  """
        if self.rootNode:
            return self.rootNode.height()
        else:
            return 0
        
    def skew(self):
        if self.rootNode is None:
            return 0
        else:
            return self.rootNode.skew()
    

    def size(self):
        """ Number of nodes in tree, including root """
        if self.rootNode:
            leftSize = BST(self.rootNode.left).size()   if self.rootNode.left else 0
            rightSize = BST(self.rootNode.right).size() if self.rootNode.right else 0
            return leftSize + rightSize + 1    
        else:
            return 0        

    def plot(self,header=None):

        if header is None:
            header = self.__str__()
            
        fig, ax = plt.subplots(1, 1, figsize=(10, 15))

        ax.set_aspect('equal')

        height = self.height()
        width = 2**(self.height()+1 )
        ratio = width/height

        nodeRadius = 0.12*ratio

        ax.set_xlim(-(width//2 - 1)*1.1, (width//2)*1.1)
        ax.set_ylim((height+2)*ratio, -ratio)
        
        
        def addNode(aNode, origin):
            if origin is None:
                origin = (0, 0)
            hspace = 2**(aNode.height()-1)


            if aNode.left:
                leftChildOrigin = (origin[0]-hspace, (origin[1]+ratio))
                line = lines.Line2D([origin[0], leftChildOrigin[0]], [
                                    origin[1], leftChildOrigin[1]], lw=1, color='black', axes=ax)
                ax.add_line(line)
                addNode(aNode.left, leftChildOrigin)
            if aNode.right:
                rightChildOrigin = (origin[0]+hspace,(origin[1]+ratio))
                line = lines.Line2D([origin[0], rightChildOrigin[0]], [
                                    origin[1], rightChildOrigin[1]], lw=1, color='black', axes=ax)
                ax.add_line(line)
                addNode(aNode.right, rightChildOrigin)
            color = getattr(aNode,'color',"black")
            if not isinstance(color,str):
                color = color.name.lower()
            ax.add_patch(plt.Circle(origin, nodeRadius,
                         color=color, ec='r', fill=True))
            plt.text(origin[0], origin[1]-1.25*nodeRadius, f"{aNode.key}")

        addNode(self.rootNode, (0, 1))

        ax.set_title(header)
        plt.show()
        
print (f"Classes redefined at time {datetime.datetime.now()}")

## Punto 1: Creación manual y display del nodo
Basico, crear nodo, árbol y crear un arbol con el nodo as root.

In [ ]:
def buildTree(keys):
    nodes = { key: Node(key,str(key)) for key in keys}
    nodes[6].connectAsLeftChildOfNode(nodes[15])
    nodes[3].connectAsLeftChildOfNode(nodes[6])
    nodes[2].connectAsLeftChildOfNode(nodes[3])
    nodes[4].connectAsRightChildOfNode(nodes[3])
    nodes[7].connectAsRightChildOfNode(nodes[6])
    nodes[13].connectAsRightChildOfNode(nodes[7])
    nodes[9].connectAsLeftChildOfNode(nodes[13])
    nodes[17].connectAsLeftChildOfNode(nodes[18])
    nodes[20].connectAsRightChildOfNode(nodes[18])
    nodes[18].connectAsRightChildOfNode(nodes[15])

    # Creo el árbol, simplemente indicándole el nodo raíz.
    bst = BST(nodes[15])
    return bst
keys = [2,3,4,6,7,9,13,15,18,17,20]
bst = buildTree(keys)
# y lo dibujo...
bst.plot()


##  Punto 2: Secuenciación Central
Añadimos un  método de secuenciaCentral de la clase BST

In [ ]:
%%add_to BST
def secuenciaCentral(self):
    """ Return the trasverse sequence  ( list of tuple (key,value)) using central order 
        lo hace de formar recursiva sobre los subarboles izquierdo, luego el nodo central, y luego el nodo derecho...
    """
    nodesList = []
    if self.rootNode.left:
        nodesList.extend(BST(self.rootNode.left).secuenciaCentral())
    nodesList.append((self.rootNode.key, self.rootNode.value))
    if self.rootNode.right:
        nodesList.extend(BST(self.rootNode.right).secuenciaCentral())
    return nodesList

Y lo ejecutamos,...

In [ ]:
print (f"La secuencia central es: {bst.secuenciaCentral()}")

-------
# ROTACIONES !!!
Implementemos el código de la rotación


## Rotación izquierda

## Probando.... reconstruyamos el árbol...

In [ ]:
bst = buildTree(keys)
bst.plot()

In [ ]:
bst.rightRotate(bst.rootNode)
bst.plot("Right  rotation in root")


Restore con una left del root

In [ ]:
rNode = bst.rootNode
print (rNode)
bst.leftRotate(rNode)
bst.plot("After left rotation on root")

Left rotate on 6 node...

In [ ]:
node6 = bst.getNodeByKey(6)
bst.leftRotate(node6, DEBUG=True)
bst.plot("after left Rotate on 6")

In [ ]:
# Hagamoslo peor... LR on 7
node7 = bst.getNodeByKey(7)
bst.leftRotate(node7, DEBUG=True)
bst.plot("after left Rotate on 7")

and correct with a rightRotate on 7

In [ ]:
node13 = bst.getNodeByKey(13)
bst.rightRotate(node13, DEBUG=True)
bst.plot("after right Rotate on 13")

In [ ]:
rnode = bst.rootNode
bst.rightRotate(rnode, DEBUG=True)
bst.plot("after right Rotate on rnode")